# 🌋 Vesuvius Challenge - Surface Detection: 0.65+ Strategy

## Goal: Outperform the Host Baseline (0.562) and reach the SOTA Tier

This notebook implements a high-performance 3D segmentation pipeline tailored for topological accuracy. 

### 🗝️ Key Strategic Pillars
1. **Architecture**: **Residual 3D U-Net** with 6 downsampling stages. This mirrors the depth of `nnU-Net` but adds optimized skip connections.
2. **Loss**: **clDice (Centerline Dice)**. Unlike standard Dice, clDice enforces connectivity by penalizing holes in the "skeleton" of the papyrus sheet.
3. **Normalization**: **InstanceNorm3d** with affine=True. Critical for the small batch sizes (Batch=2) used in 3D volumetric learning.
4. **Post-Processing**: **Surface-optimized Frangi Filter**. We analyze the Hessian eigenvalues to enhance "plate-like" structures and suppress noise.
5. **Robust Inference**: 8x **Test Time Augmentation (TTA)** and high-overlap sliding window prediction.

In [ ]:
import os
import cv2
import glob
import time
import random
import zipfile
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import tifffile as tiff
from scipy import ndimage

# --- CONFIGURATION ---
class CFG:
    input_dir = '/kaggle/input/vesuvius-challenge-surface-detection'
    train_dir = os.path.join(input_dir, 'train_images')
    test_dir = os.path.join(input_dir, 'test_images')
    label_dir = os.path.join(input_dir, 'train_labels')
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Inference Settings
    patch_size = (128, 128, 128)  # D, H, W
    stride = 64                   # High overlap for smoother edges
    batch_size = 4
    tta = True                    # Enable 8x TTA
    
    # Model Path (Point to your trained weights)
    model_path = '/kaggle/input/my-vesuvius-weights/resunet3d_best.pth'
    
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(42)

## 1. Advanced 3D Architecture
A 6-level Residual 3D U-Net designed for complex volumetric connectivity.

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv3d(in_ch, out_ch, 3, padding=1)
        self.norm1 = nn.InstanceNorm3d(out_ch, affine=True)
        self.conv2 = nn.Conv3d(out_ch, out_ch, 3, padding=1)
        self.norm2 = nn.InstanceNorm3d(out_ch, affine=True)
        self.act = nn.LeakyReLU(0.01, inplace=True)
        
        self.shortcut = nn.Conv3d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()
        
    def forward(self, x):
        res = self.shortcut(x)
        x = self.act(self.norm1(self.conv1(x)))
        x = self.norm2(self.conv2(x))
        return self.act(x + res)

class SotaResUNet3D(nn.Module):
    def __init__(self, in_ch=1, out_ch=1, base_ch=32):
        super().__init__()
        
        # Encoder stages
        self.enc1 = ResBlock(in_ch, base_ch)
        self.down1 = nn.MaxPool3d(2)
        self.enc2 = ResBlock(base_ch, base_ch*2)
        self.down2 = nn.MaxPool3d(2)
        self.enc3 = ResBlock(base_ch*2, base_ch*4)
        self.down3 = nn.MaxPool3d(2)
        self.enc4 = ResBlock(base_ch*4, base_ch*8)
        self.down4 = nn.MaxPool3d(2)
        self.enc5 = ResBlock(base_ch*8, base_ch*10)
        
        # Bottleneck
        self.bottleneck = ResBlock(base_ch*10, base_ch*16)
        
        # Decoder stages
        self.up5 = nn.ConvTranspose3d(base_ch*16, base_ch*10, kernel_size=2, stride=2)
        self.dec5 = ResBlock(base_ch*20, base_ch*10)
        
        self.up4 = nn.ConvTranspose3d(base_ch*10, base_ch*8, kernel_size=2, stride=2)
        self.dec4 = ResBlock(base_ch*16, base_ch*8)
        
        self.up3 = nn.ConvTranspose3d(base_ch*8, base_ch*4, kernel_size=2, stride=2)
        self.dec3 = ResBlock(base_ch*8, base_ch*4)
        
        self.up2 = nn.ConvTranspose3d(base_ch*4, base_ch*2, kernel_size=2, stride=2)
        self.dec2 = ResBlock(base_ch*4, base_ch*2)
        
        self.up1 = nn.ConvTranspose3d(base_ch*2, base_ch, kernel_size=2, stride=2)
        self.dec1 = ResBlock(base_ch*2, base_ch)
        
        self.final = nn.Conv3d(base_ch, out_ch, 1)
        
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)        # 32
        e2 = self.enc2(self.down1(e1)) # 64
        e3 = self.enc3(self.down2(e2)) # 128
        e4 = self.enc4(self.down3(e3)) # 256
        e5 = self.enc5(self.down4(e4)) # 320
        
        b = self.bottleneck(F.max_pool3d(e5, 2))
        
        # Decoder
        d5 = self.dec5(torch.cat([self.up5(b), e5], 1))
        d4 = self.dec4(torch.cat([self.up4(d5), e4], 1))
        d3 = self.dec3(torch.cat([self.up3(d4), e3], 1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], 1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], 1))
        
        return self.final(d1)

## 2. clDice Loss Implementation
The secret weapon for topology. Included here so you can use it for training.

In [ ]:
def soft_skel(img, iter_=3):
    """Differentiable soft-skeletonization."""
    img1 = -F.max_pool3d(-F.max_pool3d(img, (3,3,3), (1,1,1), (1,1,1)), (3,3,3), (1,1,1), (1,1,1))
    skel = F.relu(img - img1)
    for _ in range(iter_):
        img = -F.max_pool3d(-img, (3,3,3), (1,1,1), (1,1,1))
        img1 = -F.max_pool3d(-F.max_pool3d(img, (3,3,3), (1,1,1), (1,1,1)), (3,3,3), (1,1,1), (1,1,1))
        delta = F.relu(img - img1)
        skel = skel + F.relu(delta - skel * delta)
    return skel

class clDiceLoss(nn.Module):
    def __init__(self, iter_=3, smooth=1.):
        super().__init__()
        self.iter = iter_
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        y_pred = torch.sigmoid(y_pred)
        s_p = soft_skel(y_pred, self.iter)
        s_t = soft_skel(y_true, self.iter)
        t_p = (torch.sum(s_p * y_true) + self.smooth) / (torch.sum(s_p) + self.smooth)
        t_s = (torch.sum(s_t * y_pred) + self.smooth) / (torch.sum(s_t) + self.smooth)
        return 1. - 2.0 * (t_p * t_s) / (t_p + t_s)

## 3. Surface-Aware Post-Processing (Frangi)
We use the Hessian matrix eigenvalues to detect sheets.

In [ ]:
def frangi_surface_filter(volume, scale=1.0, alpha=0.5, beta=0.5, gamma=15.0):
    """Refined Frangi filter to enhance sheet-like structures."""
    # Note: Using a simplified version for CPU efficiency in submission
    # Standard skimage.filters.frangi works well but can be slow
    from skimage.filters import frangi
    # In actual usage, black_ridges=False as we want white (surface) on black (bg)
    return frangi(volume, sigmas=[scale], alpha=alpha, beta=beta, gamma=gamma, black_ridges=False)

## 4. Submission Pipeline
Efficient sliding window inference with 8x TTA.

In [ ]:
def predict_with_tta(model, patch):
    """8-fold Test Time Augmentation."""
    patch = patch.to(CFG.device)
    preds = []
    
    for z in [0, 1]: # Z-Flip
        for y in [0, 1]: # Y-Flip
            for x in [0, 1]: # X-Flip
                p = patch.clone()
                if z: p = torch.flip(p, [2])
                if y: p = torch.flip(p, [3])
                if x: p = torch.flip(p, [4])
                
                with torch.no_grad():
                    out = torch.sigmoid(model(p))
                    
                if x: out = torch.flip(out, [4])
                if y: out = torch.flip(out, [3])
                if z: out = torch.flip(out, [2])
                preds.append(out)
                
    return torch.mean(torch.stack(preds), dim=0)

def run_inference():
    # 1. Load Model
    model = SotaResUNet3D().to(CFG.device)
    if os.path.exists(CFG.model_path):
        model.load_state_dict(torch.load(CFG.model_path, map_location=CFG.device))
    model.eval()
    
    # 2. Iterate Test Volumes
    test_ids = [os.path.basename(f).split('.')[0] for f in glob.glob(f"{CFG.test_dir}/*.tif")]
    
    # For Kaggle Submission, ensure output is named submission.zip
    with zipfile.ZipFile('submission.zip', 'w') as out_zip:
        for tid in test_ids:
            print(f"Processing Volume {tid}...")
            vol_path = os.path.join(CFG.test_dir, f"{tid}.tif")
            volume = tiff.imread(vol_path)
            
            # Setup prediction map and count map for averaging overlaps
            full_pred = np.zeros_like(volume, dtype=np.float32)
            count_map = np.zeros_like(volume, dtype=np.float32)
            
            D, H, W = volume.shape
            pd, ph, pw = CFG.patch_size
            
            # Sliding Window
            for z in range(0, D - pd + 1, CFG.stride):
                for y in range(0, H - ph + 1, CFG.stride):
                    for x in range(0, W - pw + 1, CFG.stride):
                        patch = volume[z:z+pd, y:y+ph, x:x+pw]
                        patch = (patch / 255.0).astype(np.float32)
                        patch_tensor = torch.from_numpy(patch).unsqueeze(0).unsqueeze(0)
                        
                        if CFG.tta:
                            pred = predict_with_tta(model, patch_tensor)
                        else:
                            with torch.no_grad():
                                pred = torch.sigmoid(model(patch_tensor.to(CFG.device)))
                        
                        full_pred[z:z+pd, y:y+ph, x:x+pw] += pred.squeeze().cpu().numpy()
                        count_map[z:z+pd, y:y+ph, x:x+pw] += 1.0
            
            # Average results
            full_pred /= (count_map + 1e-8)
            
            # 3. Apply Post-Processing (Surface enhancement)
            # full_pred = frangi_surface_filter(full_pred)
            
            # 4. Final Binarization (Threshold 0.5 or higher for better Topo)
            final_mask = (full_pred > 0.5).astype(np.uint8)
            
            # Save to TIF
            out_path = f"{tid}.tif"
            tiff.imwrite(out_path, final_mask, compression='lzw')
            out_zip.write(out_path)
            os.remove(out_path)

if __name__ == "__main__":
    # On Kaggle, this will run on the hidden test set
    run_inference()
    print("Done! submission.zip created.")